In [4]:
import sys
sys.path.insert(0, '../../')
from questions_construction.questions import *
from questions_construction.domains import DOMAIN_NAMES
from analysis.model_performances import *
from common import *
import random
from collections import defaultdict
from copy import deepcopy

In [5]:
PLAN_LENGTHS = [19]
SMALL_MODELS = []
BIG_MODELS = ['gpt-4o']
TUNED_MODELS = []
PROMPT_MODEL_NAMES = BIG_MODELS + SMALL_MODELS + TUNED_MODELS
PROMPT_TYPES = ['few_shot_0', 'few_shot_1']
SUBSTITUTION_TYPES = [WITHOUT_RANDOM_SUB]
RAMIFICATION_TYPES = [WITHOUT_RAMIFICATIONS]

questions_dir = f'{DATA_PATH}/questions'
ids_file_name = 'dataset_ids.test.pruned'
if ids_file_name:
    selected_ids = open_jsonl(f'{DATA_PATH}/{ids_file_name}.jsonl') + open_jsonl(f'{DATA_PATH}/questions.composite.test_ids.jsonl')
    questions_by_id = gather_questions(questions_dir, selected_ids=selected_ids)
    data_all, missing_data = gather_data(questions_by_id, selected_ids=selected_ids)
    save_main_dir = f'{STATISTICS_PATH}.{ids_file_name}'

questions gathered


520it [00:01, 262.07it/s]                         

data is gathered


In [7]:
len(data_all)

12370

In [8]:
data_all[0]

{'prompt': "[DOMAIN DESCRIPTION]\n\nPicking up a block is only possible if that block is clear, on the table, and the hand is empty. By picking up that block, it makes that block not present on the table and not clear. It also leads to the block being held and makes the hand not empty. Putting down the block can only be executed if the block is being held. Putting down the block causes that block to be clear and on the table. It also causes the hand to be not holding the block and makes the hand empty. A block can be stacked on the second block if it is being held and the second block is clear. By stacking the first block on the second, it causes the first block to clear and on top of the second block. Meanwhile, the second block is not clear, and the hand becomes empty as it is not holding the block. The block can also be unstacked from the top of the second block only if the hand is empty and the first block is clear and on top of the second block. Unstacking the first block from the

In [64]:
prompt_type = 'few_shot_0'
answer_type = FREE_ANSWER_TYPE

filter_by = base_filter(WITHOUT_RAMIFICATIONS, 'gpt-4o', prompt_type, answer_type, WITHOUT_RANDOM_SUB)
filter_by.append((OUT_OBJ_QUESTION_CATEGORY, set(['action_executability', 'fluent_tracking'])))
filter_by.append((OUT_OBJ_PLAN_LENGTH, set([19])))
selected_data_all = filter_gather(data_all, filter_by)

In [65]:
selected_data_all

[{'prompt': "[DOMAIN DESCRIPTION]\n\nPicking up a block is only possible if that block is clear, on the table, and the hand is empty. By picking up that block, it makes that block not present on the table and not clear. It also leads to the block being held and makes the hand not empty. Putting down the block can only be executed if the block is being held. Putting down the block causes that block to be clear and on the table. It also causes the hand to be not holding the block and makes the hand empty. A block can be stacked on the second block if it is being held and the second block is clear. By stacking the first block on the second, it causes the first block to clear and on top of the second block. Meanwhile, the second block is not clear, and the hand becomes empty as it is not holding the block. The block can also be unstacked from the top of the second block only if the hand is empty and the first block is clear and on top of the second block. Unstacking the first block from th

In [66]:
# reasoning_chains = []
# for d in selected_data_all:
# 
#         reasoning_chains.append(d)

In [78]:
MIN_NUM_CHARS_RESPONSE = 1000
reasoning_chains = {}
for category in ['action_executability', 'fluent_tracking']:
    reasoning_chains[category] = []
    for d in selected_data_all:
        if len(d['response']) > MIN_NUM_CHARS_RESPONSE:
            if d['question_category'] == category:
                reasoning_chains[category].append(d)
    random.shuffle(reasoning_chains[category])

In [80]:
selected_reasoning_chains_by_cat = {}
for category, d_category_ls in reasoning_chains.items():
    reasoning_chains_by_domain = defaultdict(list)
    for d in d_category_ls:
        reasoning_chains_by_domain[d['domain_name']].append(d)
    
    reasoning_chains_selected = []
    d_domain_ls = []
    while any([len(v) for v in reasoning_chains_by_domain.values()]):
        for domain_name, d_domain_ls in reasoning_chains_by_domain.items():
            if d_domain_ls:
                reasoning_chains_selected.append(d_domain_ls.pop())
    selected_reasoning_chains_by_cat[category] = reasoning_chains_selected

In [83]:
for c, v in selected_reasoning_chains_by_cat.items():
    print(c, len(v))

action_executability 22
fluent_tracking 10


In [86]:
 save_jsonl([selected_reasoning_chains_by_cat['action_executability'][-1]], 'inst')

In [62]:
for category, reasoning_chains in selected_reasoning_chains_by_cat.items():
    save_jsonl(reasoning_chains, f'reasoning_chains.{category}.jsonl')